In [2]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import regularizers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pwd

'/content'

In [5]:
cd /content/drive/MyDrive/TMLC

/content/drive/MyDrive/TMLC


In [6]:
pwd

'/content/drive/MyDrive/TMLC'

In [7]:
data = pd.read_csv('icu_data.csv')
data.head()

,hospital_death,age,bmi,elective_surgery,height,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,...,isin_metabolic,isin_musculoskeletal/skin,isin_neurological,isin_respiratory,isin_sepsis,isin_trauma,isin_haematologic,isin_neurologic,isin_renal/genitourinary,isin_undefined diagnoses
0,0,68.0,22.730000,0,180.3,0.541667,73.9,113.0,502.01,0,...,0,0,0,0,1,0,0,0,0,0
1,0,77.0,27.420000,0,160.0,0.927778,70.2,108.0,203.01,0,...,0,0,0,1,0,0,0,0,0,0
2,0,67.0,27.560000,0,190.5,0.000694,100.0,301.0,403.01,0,...,0,0,1,0,0,0,0,1,0,0
3,0,46.0,25.845717,0,167.6,0.000000,72.6,108.0,203.01,0,...,0,0,0,1,0,0,0,0,0,0
4,0,87.0,21.963763,0,180.3,5.046528,71.4,113.0,501.05,0,...,0,0,0,0,1,0,0,0,0,0


In [8]:
data['hospital_death']

0        0
1        0
2        0
3        0
4        0
        ..
53108    0
53109    0
53110    0
53111    0
53112    0
Name: hospital_death, Length: 53113, dtype: int64

In [9]:
data_y = data['hospital_death']
data_x = data.drop('hospital_death',axis=1)

In [10]:
data_x.head()

,age,bmi,elective_surgery,height,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,...,isin_metabolic,isin_musculoskeletal/skin,isin_neurological,isin_respiratory,isin_sepsis,isin_trauma,isin_haematologic,isin_neurologic,isin_renal/genitourinary,isin_undefined diagnoses
0,68.0,22.730000,0,180.3,0.541667,73.9,113.0,502.01,0,0,...,0,0,0,0,1,0,0,0,0,0
1,77.0,27.420000,0,160.0,0.927778,70.2,108.0,203.01,0,0,...,0,0,0,1,0,0,0,0,0,0
2,67.0,27.560000,0,190.5,0.000694,100.0,301.0,403.01,0,0,...,0,0,1,0,0,0,0,1,0,0
3,46.0,25.845717,0,167.6,0.000000,72.6,108.0,203.01,0,0,...,0,0,0,1,0,0,0,0,0,0
4,87.0,21.963763,0,180.3,5.046528,71.4,113.0,501.05,0,0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
data_y.value_counts()

0    48461
1     4652
Name: hospital_death, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y,  test_size=0.2, shuffle = True, random_state = 8)

In [13]:
X_train.shape, X_test.shape

((42490, 129), (10623, 129))

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
X_train.head()

,age,bmi,elective_surgery,height,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,isin_african american,isin_asian,isin_caucasian,isin_hispanic,isin_native american,isin_other/unknown,isin_f,isin_m,isin_ccu-cticu,isin_csicu,isin_cticu,isin_cardiac icu,isin_micu,isin_med-surg icu,isin_neuro icu,isin_sicu,isin_cardiovascular,isin_gastrointestinal,isin_genitourinary,isin_gynecological,isin_hematological,isin_metabolic,isin_musculoskeletal/skin,isin_neurological,isin_respiratory,isin_sepsis,isin_trauma,isin_haematologic,isin_neurologic,isin_renal/genitourinary,isin_undefined diagnoses
28981,69.0,31.972318,1,170.0,2.662500,92.4,308.0,1207.01,1,0,7.0,0.60,3.0,6.0,0,3.0,127.0,91.0,28.2,1,57.0,29.0,138.0,36.8,1,14.90,67.0,41.0,67.0,41.0,91.0,80.0,73.0,57.0,73.0,57.0,29.0,12.0,100.0,92.0,127.0,94.0,127.0,94.0,37.1,36.7,62.0,55.0,62.0,55.0,91.0,90.0,73.0,68.0,73.0,68.0,23.0,12.0,99.0,98.0,127.0,105.0,127.0,105.0,37.1,36.9,8.0,7.0,8.1,8.1,0.69,0.60,127.0,105.0,20.0,20.0,13.3,8.6,39.0,27.0,245.0,227.0,4.5,3.8,140.0,138.0,16.50,11.50,0.01,0.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34168,43.0,42.292992,0,167.6,0.169444,118.8,113.0,501.05,0,0,9.0,0.51,4.0,6.0,0,5.0,184.0,101.0,42.2,0,134.0,39.0,132.0,35.8,1,13.75,123.0,78.0,123.0,78.0,100.0,41.0,134.0,89.0,134.0,89.0,47.0,16.0,98.0,82.0,181.0,125.0,181.0,125.0,36.2,35.8,112.0,93.0,112.0,93.0,91.0,59.0,127.0,106.0,127.0,106.0,26.0,19.0,98.0,89.0,172.0,135.0,172.0,135.0,36.2,36.2,9.0,9.0,8.8,8.8,0.51,0.51,207.0,147.0,26.0,26.0,13.7,13.7,42.2,42.2,294.0,294.0,4.7,4.7,132.0,132.0,13.75,13.75,0.05,0.03,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
17273,30.0,21.527821,0,152.4,-0.004167,50.0,123.0,702.01,0,0,18.0,0.76,4.0,6.0,0,5.0,298.0,139.0,40.1,0,71.0,29.0,140.0,36.7,0,12.70,81.0,57.0,81.0,57.0,134.0,93.0,101.0,71.0,101.0,71.0,29.0,14.0,100.0,96.0,144.0,111.0,144.0,111.0,38.0,36.7,63.0,63.0,63.0,63.0,117.0,117.0,81.0,81.0,81.0,81.0,27.0,27.0,100.0,100.0,132.0,132.0,132.0,132.0,36.7,36.7,18.0,6.0,7.7,6.5,0.87,0.58,330.0,69.0,17.0,7.0,14.1,12.8,40.1,36.3,205.0,168.0,4.6,3.0,143.0,140.0,12.70,8.70,-1.00,-1.00,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
19501,70.0,30.782267,0,170.8,0.227083,89.8,110.0,110.01,0,0,14.0,0.93,4.0,6.0,0,5.0,79.0,92.0,39.1,0,117.0,26.0,134.0,36.0,0,8.00,114.0,63.0,114.0,63.0,90.0,72.0,117.0,72.0,117.0,72.0,25.0,13.0,97.0,75.0,139.0,95.0,139.0,95.0,36.9,36.0,71.0,63.0,71.0,63.0,73.0,72.0,81.0,72.0,81.0,72.0,13.0,13.0,75.0,75

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)


In [20]:
X_train_new = pd.DataFrame(X_train_std, columns= X_train.columns)

In [21]:
X_test_new = pd.DataFrame(X_test_std, columns= X_test.columns)

In [22]:
X_test_new.head()

,age,bmi,elective_surgery,height,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,isin_african american,isin_asian,isin_caucasian,isin_hispanic,isin_native american,isin_other/unknown,isin_f,isin_m,isin_ccu-cticu,isin_csicu,isin_cticu,isin_cardiac icu,isin_micu,isin_med-surg icu,isin_neuro icu,isin_sicu,isin_cardiovascular,isin_gastrointestinal,isin_genitourinary,isin_gynecological,isin_hematological,isin_metabolic,isin_musculoskeletal/skin,isin_neurological,isin_respiratory,isin_sepsis,isin_trauma,isin_haematologic,isin_neurologic,isin_renal/genitourinary,isin_undefined diagnoses
0,0.794521,0.337382,0.0,0.733002,0.004253,0.457259,0.077295,0.002381,0.0,0.0,0.089431,0.048713,1.000000,1.0,0.0,1.0,0.176880,0.209459,0.846591,0.0,0.17500,0.142857,0.560976,0.592105,0.0,0.109131,0.403361,0.519481,0.403361,0.519481,0.210084,0.429530,0.314516,0.511111,0.322314,0.511111,0.141026,0.125000,0.977011,0.94,0.500000,0.588235,0.500000,0.588131,0.375000,0.780071,0.311321,0.406593,0.308411,0.402174,0.245763,0.268519,0.344828,0.405660,0.350877,0.405660,0.204082,0.124031,0.977778,0.96,0.581081,0.539007,0.581081,0.535211,0.524590,0.578125,0.090164,0.109002,0.478261,0.604167,0.045497,0.054991,0.120818,0.372549,0.535714,0.62500,0.846154,0.903509,0.823151,0.882006,0.211470,0.234645,0.238095,0.352941,0.485714,0.638889,0.102496,0.122506,0.515152,0.512690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.452055,0.383497,0.0,0.303134,0.004367,0.310448,0.053140,0.002857,0.0,0.0,0.008130,0.035846,1.000000,1.0,0.0,1.0,0.175094,0.445946,0.696023,0.0,0.47500,0.464286,0.414634,0.486842,0.0,0.316927,0.537815,0.740260,0.537815,0.740260,0.277311,0.483221,0.451613,0.711111,0.462810,0.711111,0.205128,0.145833,1.000000,0.95,0.507042,0.672269,0.507042,0.672186,0.375000,0.661648,0.688679,0.703297,0.682243,0.695652,0.381356,0.435185,0.577586,0.792453,0.587719,0.792453,0.183673,0.108527,1.000000,0.97,0.452703,0.588652,0.452703,0.584507,0.393443,0.453125,0.008197,0.018167,0.565217,0.687500,0.032498,0.040465,0.118959,0.274510,0.285714,0.40625,0.692308,0.763158,0.652733,0.725664,0.399642,0.429486,0.333333,0.529412,0.314286,0.472222,0.310383,0.355768,0.510101,0.512690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.136986,0.235238,0.0,0.652509,0.005812,0.307395,0.082126,0.239995,0.0,0.0,0.056911,0.050551,1.000000,1

In [23]:
from keras import Sequential
from keras.layers import Dense


In [24]:
X_train_new.shape[-1]

129

In [25]:
model = Sequential()

model.add(Dense(128, input_shape=(X_train_new.shape[-1],), activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16640     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 49,793
Trainable params: 49,793
Non-trainable params: 0
_________________________________________________________________


In [27]:
metrics = [
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(curve='ROC')
]


### Compiling the model
### choosing an appropirate optimiser and loss function
model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='binary_crossentropy',
              metrics=metrics)

In [28]:
### Training the model
epochs = 20
batch_size = 32
history = model.fit(X_train_new.values,y_train, validation_data = (X_test_new.values, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/20
1328/1328 [==============================] - 12s 5ms/step - loss: 0.2310 - precision: 0.6046 - recall: 0.1414 - auc: 0.8323 - val_loss: 0.2114 - val_precision: 0.6311 - val_recall: 0.2401 - val_auc: 0.8694
Epoch 2/20
1328/1328 [==============================] - 6s 5ms/step - loss: 0.2201 - precision: 0.6432 - recall: 0.2048 - auc: 0.8517 - val_loss: 0.2125 - val_precision: 0.7500 - val_recall: 0.0625 - val_auc: 0.8720
Epoch 3/20
1328/1328 [==============================] - 7s 5ms/step - loss: 0.2146 - precision: 0.6708 - recall: 0.2190 - auc: 0.8613 - val_loss: 0.2070 - val_precision: 0.6413 - val_recall: 0.2314 - val_auc: 0.8703
Epoch 4/20
1328/1328 [==============================] - 6s 5ms/step - loss: 0.2122 - precision: 0.6669 - recall: 0.2201 - auc: 0.8660 - val_loss: 0.2058 - val_precision: 0.6279 - val_recall: 0.2664 - val_auc: 0.8751
Epoch 5/20
1328/1328 [==============================] - 7s 5ms/step - loss: 0.2110 - precision: 0.6657 - recall: 0.2374 - auc: 0.8678 -

### Using Dropouts

In [29]:
model = Sequential()

model.add(Dense(128, input_shape=(X_train_new.shape[-1],), activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               16640     
                                                                 
 dense_5 (Dense)             (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 49,793
Trainable params: 49,793
Non-trai

In [31]:
metrics = [
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(curve='ROC')
]


### Compiling the model
### choosing an appropirate optimiser and loss function
model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='binary_crossentropy',
              metrics=metrics)

In [32]:
### Training the model
epochs = 20
batch_size = 32
history = model.fit(X_train_new.values,y_train, validation_data = (X_test_new.values, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/20
1328/1328 [==============================] - 8s 6ms/step - loss: 0.2493 - precision: 0.5547 - recall: 0.0570 - auc_1: 0.8065 - val_loss: 0.2166 - val_precision: 0.7742 - val_recall: 0.0789 - val_auc_1: 0.8614
Epoch 2/20
1328/1328 [==============================] - 7s 5ms/step - loss: 0.2379 - precision: 0.6107 - recall: 0.1187 - auc_1: 0.8305 - val_loss: 0.2169 - val_precision: 0.7481 - val_recall: 0.1075 - val_auc_1: 0.8610
Epoch 3/20
1328/1328 [==============================] - 7s 5ms/step - loss: 0.2351 - precision: 0.6254 - recall: 0.1487 - auc_1: 0.8301 - val_loss: 0.2271 - val_precision: 0.8462 - val_recall: 0.0362 - val_auc_1: 0.8581
Epoch 4/20
1328/1328 [==============================] - 6s 5ms/step - loss: 0.2339 - precision: 0.6190 - recall: 0.1481 - auc_1: 0.8332 - val_loss: 0.2391 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc_1: 0.8603
Epoch 5/20
1328/1328 [==============================] - 6s 5ms/step - loss: 0.2370 - precision: 0.6120 - recall:

### Dealing with Class Imbalance

In [59]:
y_train.value_counts()[0]

weight_for_0 = 1.0 / y_train.value_counts()[0] 
weight_for_1 = 1.0 / y_train.value_counts()[1]  * 5

weight_for_0, weight_for_1

(2.5806451612903226e-05, 0.0013368983957219253)

In [60]:
model = Sequential()

model.add(Dense(128, input_shape=(X_train_new.shape[-1],), activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [67]:
metrics = [
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(curve='ROC')
]


### Compiling the model
### choosing an appropirate optimiser and loss function
model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='binary_crossentropy',
              metrics=metrics)

In [68]:
callbacks = [keras.callbacks.ModelCheckpoint("patient_model_at_epoch_{epoch}.h5")]
class_weight = {0: 0.5, 1: 5}


In [69]:
### Training the model
epochs = 20
batch_size = 32
history = model.fit(X_train_new.values,y_train, validation_data = (X_test_new.values, y_test), epochs=epochs, batch_size=batch_size, callbacks=callbacks, class_weight=class_weight)

Epoch 1/20
1328/1328 [==============================] - 9s 6ms/step - loss: 0.4570 - precision: 0.2345 - recall: 0.7687 - auc_11: 0.8385 - val_loss: 0.4051 - val_precision: 0.2709 - val_recall: 0.7533 - val_auc_11: 0.8693
Epoch 2/20
1328/1328 [==============================] - 7s 5ms/step - loss: 0.4226 - precision: 0.2589 - recall: 0.7618 - auc_11: 0.8587 - val_loss: 0.3885 - val_precision: 0.3111 - val_recall: 0.6897 - val_auc_11: 0.8686
Epoch 3/20
1328/1328 [==============================] - 8s 6ms/step - loss: 0.4361 - precision: 0.2658 - recall: 0.7350 - auc_11: 0.8550 - val_loss: 1.1728 - val_precision: 0.1413 - val_recall: 0.9583 - val_auc_11: 0.8665
Epoch 4/20
1328/1328 [==============================] - 7s 6ms/step - loss: 0.4195 - precision: 0.2777 - recall: 0.7358 - auc_11: 0.8643 - val_loss: 0.6079 - val_precision: 0.2068 - val_recall: 0.8432 - val_auc_11: 0.8657
Epoch 5/20
1328/1328 [==============================] - 7s 5ms/step - loss: 0.4079 - precision: 0.2854 - recall: